<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/main/notebooks/federated_model_visual_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing DINO and installing its dependencies

In [ ]:
# @title Clon the DINO ripo
!git clone https://github.com/facebookresearch/dino.git

Cloning into 'dino'...
remote: Enumerating objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175 (from 1)
Receiving objects: 100% (175/175), 24.47 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
# @title Installing required dependencies regarding DINO
%cd dino
!pip install -r requirements.txt
!pip install timm

/content/dino
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitl


# preprocessing the CIFAR-100 dataset

feature size in CIFAR is 32x32 but DINO requires 224x224 in the input layer.

In first step we upscale the dataset and then we add randomization to it

In last step of transformation we normalize data usind mean value and standard division of ImageNet



In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split,DataLoader

In [ ]:
# transform_train = transforms.Compose([
#     transforms.Resize(256),
#     transforms.RandomCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.485, 0.456, 0.406),
#                          std=(0.229, 0.224, 0.225))
# ])

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])





In [ ]:


full_train = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)

#Decide the split sizes
train_frac = 0.8   # e.g. 80% train
val_frac   = 0.2   #     20% validation


n_total = len(full_train)                # 50 000
n_train = int(train_frac * n_total)      # 40 000
n_val   = n_total - n_train              # 10 000


train_dataset, val_dataset = random_split(full_train, [n_train, n_val])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,  num_workers=2)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2)



100%|██████████| 169M/169M [00:28<00:00, 5.84MB/s]


# Set Hyperparameters regarding the data spliting here!

In [ ]:
#@title set the parameters here!!

number_of_clients = None
train_frac = 0.8 #@param
val_frac = 0.2 #@param
is_seed_fixed = True #@param{type:"boolean"}
seed = 42 #@param{type:"integer"}

#@markdown </br></br></br>Subsets of dataset will be divided by n_rounds. It indicates the number of rounds which each client will be trained.

#@markdown </br> In each round (n_round= n) a global model will be generated by marging the models traind by clients using Nth division of each subset then by using the gnerated global model as initial model for all clients and (N+1)th division of subset round N+1 would be started
n_rounds = 2 #@param{type:"integer"}
#@markdown </br></br> <b>splitting hyperparameters</b>

spliting_method = "i.i.d. sharing" #@param["i.i.d. sharing","non-i.i.d. sharing"]





In [ ]:
#@title Set the parameters here only if <b>i.i.d. sharding</b> method had been selected!!
#@markdown Indicate the number of subset that should be generated by dividing the dataset
if spliting_method == "i.i.d. sharding":
  number_of_subsets = 10 #@param{type:"integer"}

#@markdown Number of subsets also indicates number of clients



In [ ]:
#@title Set the parameters here only if <b>non-i.i.d. sharing</b> method had been selected!!
#@markdown Nc is the number of classes that each subset can contain
if spliting_method == "non-i.i.d. sharing":
  Nc = 5 #@param{type:"integer"}
  number_of_subsets = 20 #@param{type:"integer"}
  # are_classes_overlaping = False #@param{type:"boolean"}

#@markdown <h3>If Number of classes considerd M then:</h3>
#@markdown <ul>
#@markdown   <li>Number of subsets should be:
#@markdown     <ul>
#@markdown       <li>
#@markdown         Greater than or equal to <b>\\(\frac{M}{N_c}\\)</b>
#@markdown       </li>
#@markdown       <li>
#@markdown         Less than or equal to <b>\\(\binom{M}{N_c}\\)</b> or <b>\\(\frac{M!}{N_c!(M - N_c)!}\\)</b>
#@markdown       </li>
#@markdown     </ul>
#@markdown   </li>
#@markdown   <li>
#@markdown   Minimum number_of_subsets means classes associated with one client <b>do not exist</b> in other clients
#@markdown   </li>
#@markdown   <li>
#@markdown   Larger number_of_subsets means more overlap in sets of classes associated with different clients
#@markdown   </li>
#@markdown </ul>

#@markdown </br></br><h4>Choosing minimum number of subsets means there will be No overlapping of class between clients! and Class distribution through the clients will be DISJOINT</h4>


#@markdown </br></br><h3>Combination of classes are randomly selected which suits definition of federated learning especially Cross-device federated learning</h3>



